<a href="https://colab.research.google.com/github/MasterAlC/LeRobot-Training/blob/main/lerobot/training-smolvla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤗 x 🦾: Training SmolVLA with LeRobot Notebook

Welcome to the **LeRobot SmolVLA training notebook**! This notebook provides a ready-to-run setup for training imitation learning policies using the [🤗 LeRobot](https://github.com/huggingface/lerobot) library.

In this example, we train an `SmolVLA` policy using a dataset hosted on the [Hugging Face Hub](https://huggingface.co/), and optionally track training metrics with [Weights & Biases (wandb)](https://wandb.ai/).

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=YOUR_USERNAME/YOUR_DATASET`)
- Optional: A [wandb](https://wandb.ai/) account if you want to enable training visualization
- Recommended: GPU runtime (e.g., NVIDIA A100) for faster training

## ⏱️ Expected Training Time
Training with the `SmolVLA` policy for 20,000 steps typically takes **about 5 hours on an NVIDIA A100** GPU. On less powerful GPUs or CPUs, training may take significantly longer!

## Example Output
Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`. If `wandb` is enabled, progress will also be visualized in your wandb project dashboard.


## Install conda
This cell uses `condacolab` to bootstrap a full Conda environment inside Google Colab.


In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


## Install LeRobot
This cell clones the `lerobot` repository from Hugging Face, installs FFmpeg (version 7.1.1), and installs the package in editable mode.


In [3]:
!git clone https://github.com/huggingface/lerobot.git
!conda install ffmpeg=7.1.1 -c conda-forge
!cd lerobot && pip install -e .

Cloning into 'lerobot'...
remote: Enumerating objects: 41654, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 41654 (delta 32), reused 38 (delta 28), pack-reused 41596 (from 2)
Receiving objects: 100% (41654/41654), 202.88 MiB | 25.41 MiB/s, done.
Resolving deltas: 100% (26815/26815), done.
Filtering content: 100% (45/45), 69.03 MiB | 45.58 MiB/s, done.
/bin/bash: line 1: conda: command not found
Obtaining file:///content/lerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 29.0 MB/s eta 0:00:00

## Weights & Biases login
This cell logs you into Weights & Biases (wandb) to enable experiment tracking and logging.

In [1]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alanc-za (alanc-za-university-of-pretoria) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Install SmolVLA dependencies

In [4]:
!cd lerobot && pip install -e ".[smolvla]"

Obtaining file:///content/lerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 6.8 MB/s eta 0:00:00
  Building editable for lerobot (pyproject.toml) ... done
  Created wheel for lerobot: filename=lerobot-0.4.2-0.editable-py3-none-any.whl size=15629 sha256=fbec83419ca25c57fda8e0405b382dfe5eecee35def7f4f9af361c37d7ce2383
  Stored in directory: /tmp/pip-ephem-wheel-cache-9a0adeuo/wheels/09/b4/fe/75732b1d640db96ba1f856f2b7328b232a03b696a39cb59686
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=7e29bc42195c927f170bcd780ddf50497f83f23d6c781c221d7792c47e74e16a
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built lerobot do

## Start training SmolVLA with LeRobot

This cell runs the `train.py` script from the `lerobot` library to train a robot control policy.  

Make sure to adjust the following arguments to your setup:

1. `--dataset.repo_id=YOUR_HF_USERNAME/YOUR_DATASET`:  
   Replace this with the Hugging Face Hub repo ID where your dataset is stored, e.g., `pepijn223/il_gym0`.

2. `--batch_size=64`: means the model processes 64 training samples in parallel before doing one gradient update. Reduce this number if you have a GPU with low memory.

3. `--output_dir=outputs/train/...`:  
   Directory where training logs and model checkpoints will be saved.

4. `--job_name=...`:  
   A name for this training job, used for logging and Weights & Biases.

5. `--policy.device=cuda`:  
   Use `cuda` if training on an NVIDIA GPU. Use `mps` for Apple Silicon, or `cpu` if no GPU is available.

6. `--wandb.enable=true`:  
   Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

In [ ]:
!cd lerobot && python lerobot/scripts/train.py \
  --policy.path=lerobot/smolvla_base \
  --dataset.repo_id=Alkatt/so101_CubeToBowl_PickPlace_ASN \
  --batch_size=64 \
  --steps=20000 \
  --output_dir=outputs/train/my_smolvla \
  --job_name=my_smolvla_training \
  --policy.device=cuda \
  --wandb.enable=true

## Login into Hugging Face Hub
Now after training is done login into the Hugging Face hub and upload the last checkpoint

In [ ]:
!huggingface-cli login

In [ ]:
!huggingface-cli upload ${HF_USER}/my_smolvla \
  /content/lerobot/outputs/train/my_smolvla/checkpoints/last/pretrained_model